In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from numpy.random import default_rng
pi = np.pi

In [2]:
def make_image_stack(path):
    df = pd.read_csv(path)
    labels = df['label'].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    img_stack = df.to_numpy()
    return img_stack, labels

In [3]:
def get_image_by_index(index, img_stack, dim=28):
    img = img_stack[index]
    img = img.reshape(dim,dim)
    return img

In [4]:
# Not necessary for the algorithm, but was used in tests to check the images modulation
def show_img(img):
    plt.imshow(img, cmap="gray_r")
    plt.show()    

In [5]:
def phase_modulate_img(img):
    width, height = img.shape[0], img.shape[1]

    #Perform binary modulation to phases of 0 or pi, according to threshold
    threshold = 127
    phase = np.where(img > threshold, 0, pi)
    # Convert phase to complex exponent
    modulated = np.exp(1j * phase)
    return modulated

In [10]:
def random_phase_mask(dimx=28, dimy=28):
    random_phases = np.exp(1j * 2 * pi * np.random.rand(dimx*dimy))
    return random_phases

In [11]:
def fresnel_integral(img, wavelength, z):
    # Compute the size of the input image
    height, width = img.shape
    # Compute the sampling interval
    dx = dy = 1 / img.shape[0]
    # Create a meshgrid of sampling points
    x, y = np.meshgrid(np.linspace(-0.5, 0.5-dx, width), np.linspace(-0.5, 0.5-dy, height))
    # Compute the diffraction kernel
    k = 2 * pi / wavelength
    kernel = np.exp(1j * k * (x**2 + y**2) / (2 * z)) / (1j * wavelength * z) #FIXME - might need to add another e^jkz)
    # Compute the Fourier transform of the image
    fft_img = np.fft.fft2(img)
    # Multiply the Fourier transform by the diffraction kernel
    fft_kernel = np.fft.fft2(kernel)
    fft_result = fft_img * fft_kernel
    # Compute the inverse Fourier transform of the result
    result = np.fft.ifft2(fft_result)
    # Compute the intensity of the result
    intensity = np.abs(result)**2
    return intensity

In [12]:
def intensity_comparison(result, label):
    # Get the dimensions of the result array
    M, N = result.shape

    # Calculate the intensity in the upper half of the result array
    upper_intensity = np.sum(np.abs(result[:M//2, :])**2)

    # Calculate the intensity in the lower half of the result array
    lower_intensity = np.sum(np.abs(result[M//2:, :])**2)
    
    #Check if classification was performed correctly
    if ((upper_intensity>lower_intensity and label == 0) or (upper_intensity<lower_intensity and label ==1)):
        return 1
    else:
        return 0

In [13]:
def fitness(mask, img_arr, labels, wavelength, z):
    
    score = 0
    
    for i in range(img_arr.shape[0]):
        img = get_image_by_index(i, img_arr)
        modulated = phase_modulate_img(img)
        mask = mask.reshape((28,28))
        phase_shifted_image = modulated * mask
        intensity = fresnel_integral(phase_shifted_image, 10**-7, 100)
        score += intensity_comparison(intensity, labels[i])
    
    return score


In [34]:
def genetic_algorithm(wavelength, z, pic_dim=28, pop_size=20, num_generations=50, mutation_prob=0.1):
    
    #Load training data
    train_arr, labels = make_image_stack("mnist_train_cleaned.csv")

    rng = default_rng()
    
    population = np.empty([pop_size,pic_dim**2], dtype=complex)
    for i in range(population.shape[0]):
        population[i] = random_phase_mask(28,28) #FIXME
            
    # Loop through generations
    for i in range(num_generations):
        # Evaluate fitness for each individual in the population
        fitness_values = [fitness(mask, train_arr, labels, wavelength, z) for mask in population]
        # Select the top 50% performers to be the parents
        num_parents = pop_size // 2
        sorted_indices = np.argsort(fitness_values)
        parents = population[sorted_indices[-num_parents:]]
        # Generate offspring using crossover and mutation
        offspring = np.zeros_like(parents)
        for j in range(num_parents):
            # Crossover: take half the genes from each parent
            parent1 = parents[j]
            parent2 = parents[np.random.randint(num_parents)]
            crossover_point = rng.integers(pic_dim**2)
            offspring[j, :crossover_point] = parent1[:crossover_point]
            offspring[j, crossover_point:] = parent2[crossover_point:]
            # Mutation: randomly shift some masking elements according to mutation probability
            mutation_mask = rng.random(size=pic_dim**2) < mutation_prob
            offspring[j][mutation_mask] = (np.exp(1j * 2 * pi * np.random.random()))*offspring[j][mutation_mask]
        # Combine parents and offspring to form new population
        population = np.vstack((parents, offspring))
        
    # Return the best mask found
    best_index = np.argmax(fitness_values)
    best_mask = population[best_index].reshape(pic_dim,pic_dim)
    return best_mask

In [24]:
def check_result(mask):
    test_arr, labels = make_image_stack("mnist_test_cleaned.csv")
    
    score = fitness(mask, test_arr, labels, 10**-7, 100)
    print(f"The mask classifies correctly with {100*score/test_arr.shape[0]}%")


In [26]:
#z has to signifcantly larger than x,y, see fresnel function. FIXME - smaller z might be enough?
best = genetic_algorithm(wavelength=10**-7, z=100) 

In [36]:
check_result(best)

The mask classifies correctly with 98.81796690307328%
